## This Notebook shows how a raw Dataset gets processed for further machine learning steps

In [3]:
# Imports
import os, sys
import numpy as np
from typing import Tuple, List, Dict

# to enable local imports
module_path = os.path.abspath(os.path.join('code'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
from utils import readFileCSV, loadConfigFile
from pipelines import (filter_signal, pre_process_signal, feature_extraction, convert_data)
import pandas as pd
from typing import Dict


def saveFeatureListToFile(featureList : List, filepath : str):
    
    if type(featureList) is not list:
        raise Exception("The given feature list is not a list!")
    
    print("Saving a feature list to: '{}'".format(filepath))
    
    f = open(filepath, "w")
    for feature in featureList:
        line = "{}\n".format(feature)
        f.write(line)
    f.close()
    

def processRawFileWithPipeline(filepath : str, yamlConfig) -> (pd.Series, pd.DataFrame, List[str]):
    ''' Process a given filepath with the current pipelines 
    
    This creates two different data objects:
        epochSeries: This is a panda.Series which contains dataframes. Each index at the series represens one epoch
        frequencyFreatureDf: This is a dataframe of the frequency features of the epochSeries. The index represnts the epochs. The features are the columns
    '''
    if not os.path.isfile(filepath):
        raise Exception("The file '{}' does not exists!".format(filepath))
    
    df = readFileCSV(filepath)
    df, channelNameList =  convert_data(df=df, config=yamlConfig, starttime=None)
    df = filter_signal(df=df, config=yamlConfig) # general filtering
    epochSeries = pre_process_signal(df=df, config=yamlConfig)   # pre-processing
    frequencyFeatureDf = feature_extraction(epochSeries=epochSeries, config=yamlConfig) # extract features
    
    return epochSeries, frequencyFeatureDf, channelNameList

def safeAndProcessRawFileWithPipeline(rawFilePath : str, fileDir : str, label : str, yamlConfig):
    ''' Process the given rawfilePath and safe the result as pickle files
    This function calls 'processRawFileWithPipeline()' and the two returning data objects will be safed
    
    @param str rawFilePath: path to file which gets process
    @param str fileDir: Directory where the data objects should be stored
    @param str label: A label to know which data we process, e.g. fatigue, normal or awake data
    @param yamlConfig: A loaded yaml config file for processing the data
    '''
    print ("Starting to process {}...".format(rawFilePath))
    # process the file
    epochSeries, frequencyFeatureDf, channelNameList = processRawFileWithPipeline(filepath=rawFilePath, yamlConfig=yamlConfig)
    
    # save the epoch series
    epochSeries.to_pickle(os.path.join(fileDir,'epochSeries_{}.pkl'.format(label)))
    
    # save the frequency df
    frequencyFeatureDf.to_pickle(os.path.join(fileDir,'frequencyFeaturesDf_{}.pkl'.format(label)))
    
    # save the channel name list
    saveFeatureListToFile(featureList=channelNameList,
                          filepath=os.path.join(fileDir, "features_channel_names.txt"))
    
    # save frequency features
    saveFeatureListToFile(featureList=list(frequencyFeatureDf.columns),
                          filepath=os.path.join(fileDir, "features_frequency_df.txt"))

def processRawDatasetToPickleFiles(datasetDirPath : str, device : str, awakeFileName : str,
                                   fatigueFileName : str, normalFileName : str, unlabeledFileName : str):
    '''
    @param str datasetDirPath: Path where the directory of the dataset is
    @param str device: name of the device, to load the correct yaml file for processing
    
    Depending on the dataset there might be awake, normal, fatigue or unlabeled data. 
    @param awakeFileName: filename of the awake data or None then it will be ignored
    @param fatigueFileName: filename of the fatigue data or None then it will be ignored
    @param normalFileName: filename of the normal data or None then it will be ignored
    @param unlabeledFileName: filename of the unlabeled data or None then it will be ignored
    '''
    
    if not os.path.isdir(datasetDirPath):
        raise Exception("The given dir path '{}' does not exist!".format(datasetDirPath))
        
    # Load the yaml config file for the processing
    yamlConfig = loadConfigFile(device)
    
    for root, dirs, files in os.walk(datasetDirPath):
        for subjectDir in dirs:
            print("#############################################")
            print("Process Subject {} Data...".format(subjectDir))
            print("---------------------------------------------")
            
            if awakeFileName is not None: 
                safeAndProcessRawFileWithPipeline(rawFilePath=os.path.join(root, subjectDir, awakeFilename),
                                                  fileDir=os.path.join(root, subjectDir),
                                                  label = "awake",
                                                  yamlConfig=yamlConfig)
                
            if fatigueFileName is not None: 
                safeAndProcessRawFileWithPipeline(rawFilePath=os.path.join(root, subjectDir, fatigueFileName),
                                                  fileDir=os.path.join(root, subjectDir),
                                                  label = "fatigue",
                                                  yamlConfig=yamlConfig)
                
            if normalFileName is not None: 
                safeAndProcessRawFileWithPipeline(rawFilePath=os.path.join(root, subjectDir, normalFileName),
                                                  fileDir=os.path.join(root, subjectDir),
                                                  label = "normal",
                                                  yamlConfig=yamlConfig)
                
            if unlabeledFileName is not None: 
                safeAndProcessRawFileWithPipeline(rawFilePath=os.path.join(root, subjectDir, normalFileName),
                                                  fileDir=os.path.join(root, subjectDir),
                                                  label = "unlabeled",
                                                  yamlConfig=yamlConfig)
    
    print("#######################################")
    print("Done processing and saving a complete Dataset!")

def loadPickeldData(dataDir : str, label : str):
    ''' Load the epochseries and frequency feature df
    
    @param str dataDir: Directory where the data is
    @param str label: decide which 
    '''
    try:
        epochSeries = pd.read_pickle(os.path.join(dataDir,'epochSeries_{}.pkl'.format(label)))
    except Exception as e:
        #print (e)
        epochSeries = None
        
    try:
        frequencyFeatureDf = pd.read_pickle(os.path.join(dataDir,'frequencyFeaturesDf_{}.pkl'.format(label)))
    except Exception as e:
        #print (e)
        frequencyFeatureDf = None

    return epochSeries, frequencyFeatureDf

def loadPickeldDataset(datasetDirPath : str) -> Dict:
    ''' This functions loads a complete dataset into a dict
    
    Each Subject contains a dict with 'awake', 'normal', 'fatigue' and 'unlabeled' entry.
    Each entry contain the epochSeries and frequencyFeatureDf
    '''
    
    if not os.path.isdir(datasetDirPath):
        raise Exception("The given dir path '{}' does not exist!".format(datasetDirPath))
    
    datasetDict = {}
    
    for root, dirs, files in os.walk(datasetDirPath):
        for subjectDir in dirs:
            print("Load Subject {} Data...".format(subjectDir))
            
            epochSeries_awake, frequencyFeatureDf_awake = loadPickeldData(dataDir = os.path.join(datasetDirPath, subjectDir),
                                                                          label="awake")
            
            epochSeries_normal, frequencyFeatureDf_normal = loadPickeldData(dataDir = os.path.join(datasetDirPath, subjectDir),
                                                                          label="normal")
            
            epochSeries_fatigue, frequencyFeatureDf_fatigue = loadPickeldData(dataDir = os.path.join(datasetDirPath, subjectDir),
                                                                          label="fatigue")
            
            epochSeries_unlabeled, frequencyFeatureDf_unlabeled = loadPickeldData(dataDir = os.path.join(datasetDirPath, subjectDir),
                                                                          label="unlabeled")
            
            datasetDict[subjectDir] = {"awake" : (epochSeries_awake, frequencyFeatureDf_awake),
                                       "normal" : (epochSeries_normal, frequencyFeatureDf_normal),
                                       "fatigue" : (epochSeries_fatigue, frequencyFeatureDf_fatigue),
                                       "unlabeled" : (epochSeries_unlabeled, frequencyFeatureDf_unlabeled)}
    return datasetDict


In [5]:
from consts import DEVICES_NEUROSCAN

PROCESS_DATA = False

if PROCESS_DATA:
    # Process the online EEG Data
    processRawDatasetToPickleFiles(datasetDirPath = "D:/Masterthesis/EEG_Data/eeg_data_online",
                              device = DEVICES_NEUROSCAN,
                              awakeFileName = None,
                              fatigueFileName = "Fatigue_state_256hz.csv",
                              normalFileName = "Normal_state_256hz.csv",
                              unlabeledFileName = None)
else:
    print ("Already processed the EEG Online Data")

Already processed the EEG Online Data


In [6]:
eegDataset = loadPickeldDataset(datasetDirPath= "D:/Masterthesis/EEG_Data/eeg_data_online")

Load Subject 1 Data...
Load Subject 10 Data...
Load Subject 11 Data...
Load Subject 12 Data...
Load Subject 2 Data...
Load Subject 3 Data...
Load Subject 4 Data...
Load Subject 5 Data...
Load Subject 6 Data...
Load Subject 7 Data...
Load Subject 8 Data...
Load Subject 9 Data...


In [11]:
def convertFeatureDfToXy(featureDf : pd.DataFrame, target : int) -> (np.ndarray, np.ndarray):
    samples = []
    targetArray = []
    
    for index, row in featureDf.iterrows():
        timesteps = []
        features = row.to_numpy() # features
        timesteps.append(features)
        samples.append(timesteps)
        
        targetArray.append(target)
    
    X = np.array(samples)
    y = np.array(targetArray)
    
    return X, y 
    

def createXyFromDataSeries(dataSeries : pd.Series, target : int) -> (np.ndarray, np.ndarray):
    ''' Create X and y for machine learning
    
    @param pd.Series dataSeries: Should be a series of dataframes
    
    X should look like this [samples, timesteps, features]
        samples: The epoch
        timesteps: E.g. if the epoch contains 200 values then the timestep should contain 200 values
        features: The actual value
    
    y should look tlike this [classIds] 
        classIds: The label for the sample of the X Data
    '''
    
    samples = []
    targetArray = []
    
    if dataSeries is None:
        raise TypeError("Data Series is None!")
    
    if type(dataSeries) != pd.Series:
        raise Exception("The given dataSeries is not a pd.Series! It is {}".format(type(dataSeries)))
    
    # loop through the data Series
    for df in dataSeries:
        
        if type(df) != pd.DataFrame: # check the type
            raise Exception("The dataseries contains a {} object - The series should dataframes only!".format(type(df)))
            
        timesteps = []
            
        for index, row in df.iterrows():
            features = row.to_numpy() # features
            timesteps.append(features)
        
        samples.append(timesteps)
        targetArray.append(target)
    
    
    X = np.array(samples)
    y = np.array(targetArray)
    
    return X, y

def createMachineLearningDataset(eegDataset, targetLabelDict) -> (np.array, np.array):
    
    X = None
    y = None
    
    for subject in eegDataset:
        for target in targetLabelDict:
            
            try:
                print("Processing Subject {} - Target: {} ...".format(subject, target))
                eegDataset[subject][target][0]
                tempX, tempy = createXyFromDataSeries(dataSeries = eegDataset[subject][target][0],
                                       target = targetLabelDict[target])

                if X is None:
                    X = tempX
                else:
                    X = np.concatenate((X, tempX))

                if y is None:
                    y = tempy
                else:
                    y = np.concatenate((y, tempy))
            
            except TypeError:
                print("Skipping Target: {}".format(target))
    
    print("Done!")
    return X,y

def saveDictToFile(myDict, filepath : str):
    ''' Save a dictionary the given file path'''
    
    print("Saving dict to {}".format(filepath))
    f = open(filepath, "w")
    for key, value in myDict.items():
        line = "{v} {k}\n".format(v=value, k=key.upper())
        f.write(str(line))
    f.close()



def createAndSafeMlDataset(eegDataset : Dict[str, Dict[str ,Tuple[pd.Series, pd.DataFrame]]], targetLabelDict : Dict,
                           dirPath : str) -> (np.array, np.array):
    
    if not os.path.isdir(dirPath):
        raise Exception("The given directory path is not valid! Given path: {}".format(dirPath))
    
    print("Creating Machine Learning Dataset!")
    X, y = createMachineLearningDataset(eegDataset, targetLabelDict)
    
    #Todo - Build the feature df somehow into that too
    
    print("\nSaving Machine Learning Dataset into this directory: {}".format(dirPath))
    np.save(os.path.join(dirPath, "data_series_X.npy"), X)
    np.save(os.path.join(dirPath, "data_series_y.npy"), y)
    
    # Save target labels
    saveDictToFile(targetLabelDict, filepath=os.path.join(dirPath,'target_labels.txt'))
    
    return X, y

#createDatasetXy(dataSeries)

In [12]:
targetLabelDict = {"normal" : 0,
                   "fatigue" : 1,
              "awake" : 2,
              "unlabeled" : 3}

CREATE_ML_DATA = False

#X, y = createMachineLearningDataset(eegDataset, targetLabelDict)
# # Todo epoch series UND frequency df X und y erstellen!!

if CREATE_ML_DATA:
    createAndSafeMlDataset(eegDataset=eegDataset,
                           targetLabelDict=targetLabelDict,
                           dirPath="D:/Masterthesis/EEG_Data/eeg_data_online")
else:
    print("Already created ML Data")


Already created ML Data


In [17]:
def loadOnlineEEGdata(test_size=0.3, shuffle=False):
    dirPath = "D:/Masterthesis/EEG_Data/eeg_data_online"
    
    # load array
    X = np.load(os.path.join(dirPath, 'data_series_X.npy'))
    y = np.load(os.path.join(dirPath, 'data_series_y.npy'))
    
    # load target labels
    
    # load feature names
    
    # Split dataset into training set and test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size,random_state=109, shuffle=shuffle) # 70% training and 30% test
    
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    
    print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
    
    return X_train, y_train, X_test, y_test

In [18]:
X_train, y_train, X_test, y_test = loadOnlineEEGdata()

(5024, 512, 40) (5024, 2) (2154, 512, 40) (2154, 2)


In [15]:
# Import train_test_split function
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM

In [19]:
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]

In [20]:
model = Sequential()
model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 0, 15, 64
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    print("n_timesteps: {} - n_features: {} - n_outputs: {}".format(n_timesteps, n_features, n_outputs))
    model = Sequential()
    model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

# summarize scores
def summarize_results(scores):
	print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

# run an experiment
def run_experiment(repeats=10):
    # load data
    trainX, trainy, testX, testy = loadOnlineEEGdata()
    print("Loaded the Dataset")
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model(trainX, trainy, testX, testy)
        score = score * 100.0
        print('>#%d: %.3f' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)


In [ ]:
run_experiment(repeats=5)

(5024, 512, 40) (5024, 2) (2154, 512, 40) (2154, 2)
Loaded the Dataset
n_timesteps: 512 - n_features: 40 - n_outputs: 2


11-May-20 15:41:29 | WARNING | From C:\Users\reiss\Anaconda3\envs\ml\lib\site-packages\keras\backend\tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

